In [21]:
import pandas as pd
# pd.set_option('max_colwidth', 500)

siads_csv = pd.read_csv('SIADS_RelatorioBalancoFisicoFinanceiro.csv', encoding='utf-8')

uasg = ''
uasg_nome = ''
ug = ''
ug_nome = ''
ourg = ''

infos = siads_csv['26418 - INST.FED.DE EDUC.,CIENC.E TEC.DE PERNAMBUCO'].drop_duplicates()
infos = infos.dropna()

uasg = infos[0].split('-')
uasg = uasg[0].strip()

uasg_nome = infos[0].split('-')
uasg_nome = uasg_nome[-1]
uasg_nome = uasg_nome.strip()

ug = infos[1].split('-')
ug = ug[0].strip()

ug_nome = infos[1].split('-')
ug_nome = ug_nome[-1].strip()

siads_df = pd.DataFrame(columns=['uasg', 'uasg_nome', 'ug', 'ug_nome', 'conta', 'conta_nome', 'subitem', 'subitem_nome', 'uorg', 
                                 'codigo', 'descricao', 'qtd', 'valor', 'valor_total'])
new_row = {
    'uasg':uasg,
    'uasg_nome':uasg_nome,
    'ug':ug,
    'ug_nome':ug_nome,
    'conta':'',
    'conta_nome':'',
    'subitem':'',
    'subitem_nome':'',
    'uorg':'',
    'codigo':'',
    'descricao':'',
    'qtd':0,
    'valor':0.0,
    'valor_total': 0.0
}


print(len(siads_csv))

siads_csv = siads_csv[~siads_csv['Unnamed: 1'].str.contains('Descrição Complementar:', na=False)]
# siads_csv = siads_csv[siads_csv['Unnamed: 1'].notna()]

# siads_csv.to_clipboard()
siads_csv = siads_csv[['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 7']]

siads_csv = siads_csv[~siads_csv['Unnamed: 0'].str.contains('CONTA', na=False)]
siads_csv = siads_csv[~siads_csv['Unnamed: 0'].str.contains('EMITIDO POR:', na=False)]
siads_csv = siads_csv[~siads_csv['Unnamed: 0'].str.contains('TOTAL  - ', na=False)]
siads_csv = siads_csv[~siads_csv['Unnamed: 0'].str.contains('TOTAL - ', na=False)]
siads_csv = siads_csv.dropna(how='all')

siads_csv = siads_csv.reset_index(drop=True)

# siads_csv = siads_csv.reset_index(drop=True)

for i in range(0, len(siads_csv)):
# for i in range(176, 180):
    
    if pd.notna(siads_csv.loc[i, 'Unnamed: 0']):
        
        conta = siads_csv.loc[0, 'Unnamed: 0'].split('-')
        new_row['conta'] = conta[0].strip()
        new_row['conta_nome'] = conta[1].strip()

        if (siads_csv.loc[i, 'Unnamed: 0'] != '115610100 - MATERIAIS DE CONSUMO'):
            subitem = siads_csv.loc[i, 'Unnamed: 0'].split(' - ')
            if len(subitem) > 1:
                new_row['subitem'] = subitem[0]
                new_row['subitem_nome'] = subitem[1]

    if pd.notna(siads_csv.loc[i, 'Unnamed: 1']):
        cods = siads_csv.loc[i, 'Unnamed: 1'].split(' - ')
        if len(cods) >= 2:  # Certifique-se de que há pelo menos código e descrição
            new_row['codigo'] = cods[0].strip()
            # Combine os demais segmentos como descrição, excluindo as partes de unidade ou quantidade
            new_row['descricao'] = ' - '.join(cods[1:-2]).strip() if len(cods) > 3 else cods[1].strip()
        qtd_str = siads_csv.loc[i + 1, 'Unnamed: 7'].replace(',', '.')
        new_row['qtd'] = int(float(qtd_str))
        valor_str = siads_csv.loc[i + 2, 'Unnamed: 7'].replace(',', '')
        new_row['valor_total'] = float(valor_str)
        if new_row['qtd'] != 0:
            new_row['valor'] = round(new_row['valor_total'] / new_row['qtd'], 2)
        else:
            new_row['valor'] = 0  # ou outro valor padrão para casos de divisão por zero

    siads_df = pd.concat([siads_df, pd.DataFrame([new_row])], ignore_index=True)

siads_df = siads_df.dropna(subset=['descricao'])
siads_df = siads_df.drop_duplicates(subset=['descricao'], keep='first').reset_index(drop=True)
siads_df = siads_df[siads_df['qtd'] != 0].reset_index(drop=True)
siads_df = siads_df.reset_index(drop=True)

siads_df.to_clipboard(index=False)
        
        # print(siads_csv.loc[i, 'Unnamed: 1'])
    
        # print(f"Linha {i}, Tamanho: {len(siads_csv.loc[i, 'Unnamed: 0'])}")
display(siads_df.info())


2210


C:\Users\Ricar\AppData\Local\Temp\ipykernel_33808\3939397716.py:96: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  siads_df = pd.concat([siads_df, pd.DataFrame([new_row])], ignore_index=True)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 417 entries, 0 to 416
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   uasg          417 non-null    object 
 1   uasg_nome     417 non-null    object 
 2   ug            417 non-null    object 
 3   ug_nome       417 non-null    object 
 4   conta         417 non-null    object 
 5   conta_nome    417 non-null    object 
 6   subitem       417 non-null    object 
 7   subitem_nome  417 non-null    object 
 8   uorg          417 non-null    object 
 9   codigo        417 non-null    object 
 10  descricao     417 non-null    object 
 11  qtd           417 non-null    object 
 12  valor         417 non-null    float64
 13  valor_total   417 non-null    float64
dtypes: float64(2), object(12)
memory usage: 45.7+ KB


None

In [23]:
totalizacao = siads_df['valor_total'].sum()
print(totalizacao)

293582.26


In [27]:
subitem_total = pd.DataFrame()
subitem_total['subitem'] = siads_df['subitem'].drop_duplicates().reset_index(drop=True)
somatorio = []
sub_siads_one = siads_df[siads_df['subitem'].str.contains(f'{subitem_total['subitem'][3]}')]
# sub_siads_one.to_clipboard()
# display(sub_siads)
# print(subitem_total['subitem_nome'][0])
for i in range(0, len(subitem_total)):
    sub_siads = siads_df[siads_df['subitem'].str.contains(f'{subitem_total['subitem'][i]}')]
    soma = sub_siads['valor_total'].sum()
    somatorio.append(soma)
    print(subitem_total['subitem'][i])
    # print(subitem_total.iloc[i])
# display(subitem_total)
subitem_total['total'] = somatorio
display(subitem_total)

14
16
17
19
21
22
23
24
25
26
28
35
36
42
44
59


,subitem,total
0,14,26274.49
1,16,57270.73
2,17,13094.92
3,19,1286.14
4,21,6680.69
5,22,24481.13
6,23,64356.63
7,24,16305.50
8,25,370.00
9,26,32565.29
